<a href="https://colab.research.google.com/github/betsyxpacheco/Welcome/blob/main/lab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 4487 Module 5 Script

## Objectives

a. Understand model accuracy.  Why is it a performance metric for classification and not regression?
    
b. Calculate accuracy for a simple majority class model (this is the same as calculating the proportion of the majority class in a binary variable). Consider: x <- c(1, 1, 1, 0, 0).  What is the majority class? What is the proportion of the majority class in x?
    
c. Fit a tree model of the target with just one predictor variable and calculate the accuracy of this model.
    
d. Interpret a tree model, and calculate information gain.
    
e. Fit a tree model of the target using all the predictors, then:  create a visualization of the tree and identify the top 3 most important predictors in this model.
    
f. How do these models compare to majority class prediction?
    
g. How will you use a classification model as part of a solution to the AdviseInvest case?

We will use the MegaTelCo data for this demonstration.

##Load Libraries

In this class we will be using 
- Pandas
- Scikitlearn
- Matplotlib


In [ ]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import csv

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree


## Getting data into Pandas

In this case we will load data from the statsmodels.org library

See the Canvas assignments and lectures for a description of the Megatelco data

**Note:** you will need to enter a code supplied by Google in the next step. 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

df = pd.read_csv (r'/content/gdrive/MyDrive/Colab Notebooks/daily_aqi_by_county_2021.csv',quoting=csv.QUOTE_NONE, encoding='utf-8')

Mounted at /content/gdrive


In [ ]:
#look at the top rows
df.head(10) 

,"""State Name""","""county Name""","""State Code""","""County Code""","""Date""","""AQI""","""Category""","""Defining Parameter""","""Defining Site""","""Number of Sites Reporting"""
0,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-01""",27,"""Good""","""PM2.5""","""01-003-0010""",1
1,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-04""",47,"""Good""","""PM2.5""","""01-003-0010""",1
2,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-07""",24,"""Good""","""PM2.5""","""01-003-0010""",1
3,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-10""",39,"""Good""","""PM2.5""","""01-003-0010""",1
4,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-13""",46,"""Good""","""PM2.5""","""01-003-0010""",1
5,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-16""",21,"""Good""","""PM2.5""","""01-003-0010""",1
6,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-19""",52,"""Moderate""","""PM2.5""","""01-003-0010""",1
7,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-22""",11,"""Good""","""PM2.5""","""01-003-0010""",1
8,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-25""",39,"""Good""","""PM2.5""","""01-003-0010""",1
9,"""Alabama""","""Baldwin""","""01""","""003""","""2021-01-28""",22,"""Good""","""PM2.5""","""01-003-0010""",1


In [ ]:
#look at the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218196 entries, 0 to 218195
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   "State Name"                 218196 non-null  object
 1   "county Name"                218196 non-null  object
 2   "State Code"                 218196 non-null  object
 3   "County Code"                218196 non-null  object
 4   "Date"                       218196 non-null  object
 5   "AQI"                        218196 non-null  int64 
 6   "Category"                   218196 non-null  object
 7   "Defining Parameter"         218196 non-null  object
 8   "Defining Site"              218196 non-null  object
 9   "Number of Sites Reporting"  218196 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 16.6+ MB


In [ ]:
#describe the data before cleaning it
df.describe()

,"""AQI""","""Number of Sites Reporting"""
count,218196.000000,218196.000000
mean,39.584250,1.924220
std,21.547055,2.251967
min,0.000000,1.000000
25%,29.000000,1.000000
50%,38.000000,1.000000
75%,47.000000,2.000000
max,2723.000000,34.000000


## Clean up the data
Did you notice anything unusual about the "house" amounts? 

How about the handset price and income?

Clean up the data in a  new datafram named "df_clean"


In [ ]:
#delete rows with outlier data; put it in a new dataframe
df_clean = df[(df['"State Name"'] == '"Indiana"')]

#delete any rows with missing values in the clean dataframe
df_clean = df_clean.dropna() 

df_clean.describe()

,"""AQI""","""Number of Sites Reporting"""
count,9051.000000,9051.000000
mean,39.858358,1.728759
std,17.000073,1.197378
min,0.000000,1.000000
25%,31.000000,1.000000
50%,38.000000,1.000000
75%,47.000000,2.000000
max,169.000000,7.000000


# Standardize attributes

- Convert the "college" and "considering_change_of_plan" attributes to 0 or 1
- Convert non-numeric attributes to numeric scale of 1-3


In [ ]:
#Get distinct values
df_clean['"State Name"'].unique() 

array(['"Indiana"'], dtype=object)

In [ ]:
df_clean['"county Name"'] .unique()

array(['"Allen"', '"Bartholomew"', '"Boone"', '"Brown"', '"Carroll"',
       '"Clark"', '"Daviess"', '"Delaware"', '"Dubois"', '"Elkhart"',
       '"Floyd"', '"Gibson"', '"Greene"', '"Hamilton"', '"Hendricks"',
       '"Henry"', '"Howard"', '"Knox"', '"LaPorte"', '"Lake"',
       '"Madison"', '"Marion"', '"Monroe"', '"Perry"', '"Porter"',
       '"Posey"', '"Shelby"', '"Spencer"', '"St. Joseph"', '"Tippecanoe"',
       '"Vanderburgh"', '"Vigo"', '"Wabash"', '"Warrick"', '"Whitley"'],
      dtype=object)

In [ ]:
df_clean['"Category"'].unique()

array(['"Good"', '"Moderate"', '"Unhealthy for Sensitive Groups"',
       '"Unhealthy"'], dtype=object)

In [ ]:
df_clean['"Defining Parameter"'].unique()

array(['"PM2.5"', '"Ozone"', '"SO2"', '"PM10"', '"NO2"', '"CO"'],
      dtype=object)

In [ ]:
df_clean.loc[df_clean['"Category"'] == '"Good"', '"Category"'] = "4"
df_clean.loc[df_clean['"Category"'] == '"Moderate"', '"Category"'] = "3"
df_clean.loc[df_clean['"Category"'] == '"Unhealthy for Sensitive Groups"', '"Category"'] = "2"
df_clean.loc[df_clean['"Category"'] == '"Unhealthy"', '"Category"'] = "1"


df_clean.loc[df_clean['"Defining Parameter"'] == '"PM2.5"', '"PM2.5"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"PM2.5"', '"PM2.5"'] = "0"

df_clean.loc[df_clean['"Defining Parameter"'] == '"Ozone"', '"Ozone"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"Ozone"', '"Ozone"'] = "0"

df_clean.loc[df_clean['"Defining Parameter"'] == '"SO2"', '"SO2"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"SO2"', '"SO2"'] = "0"

df_clean.loc[df_clean['"Defining Parameter"'] == '"PM10"', '"PM10"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"PM10"', '"PM10"'] = "0"

df_clean.loc[df_clean['"Defining Parameter"'] == '"NO2"', '"NO2"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"NO2"', '"NO2"'] = "0"

df_clean.loc[df_clean['"Defining Parameter"'] == '"CO"', '"CO"'] = "1"
df_clean.loc[df_clean['"Defining Parameter"'] != '"CO"', '"CO"'] = "0"

df_clean['"Category"'] = df_clean['"Category"'].astype('int')

df_clean['"PM2.5"'] = df_clean['"PM2.5"'].astype('int') 
df_clean['"Ozone"'] = df_clean['"Ozone"'].astype('int') 
df_clean['"SO2"'] = df_clean['"SO2"'].astype('int')
df_clean['"PM10"'] = df_clean['"PM10"'].astype('int')
df_clean['"NO2"'] = df_clean['"NO2"'].astype('int')
df_clean['"CO"'] = df_clean['"CO"'].astype('int')


df_clean.head(10) 

,"""State Name""","""county Name""","""State Code""","""County Code""","""Date""","""AQI""","""Category""","""Defining Parameter""","""Defining Site""","""Number of Sites Reporting""","""PM2.5""","""Ozone""","""SO2""","""PM10""","""NO2""","""CO"""
54477,"""Indiana""","""Allen""","""18""","""003""","""2021-01-01""",29,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54478,"""Indiana""","""Allen""","""18""","""003""","""2021-01-02""",20,4,"""Ozone""","""18-003-0002""",2,0,1,0,0,0,0
54479,"""Indiana""","""Allen""","""18""","""003""","""2021-01-03""",22,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54480,"""Indiana""","""Allen""","""18""","""003""","""2021-01-04""",31,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54481,"""Indiana""","""Allen""","""18""","""003""","""2021-01-05""",43,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54482,"""Indiana""","""Allen""","""18""","""003""","""2021-01-06""",26,4,"""Ozone""","""18-003-0002""",2,0,1,0,0,0,0
54483,"""Indiana""","""Allen""","""18""","""003""","""2021-01-07""",27,4,"""Ozone""","""18-003-0002""",2,0,1,0,0,0,0
54484,"""Indiana""","""Allen""","""18""","""003""","""2021-01-08""",28,4,"""Ozone""","""18-003-0002""",2,0,1,0,0,0,0
54485,"""Indiana""","""Allen""","""18""","""003""","""2021-01-09""",30,4,"""Ozone""","""18-003-0004""",2,0,1,0,0,0,0
54486,"""Indiana""","""Allen""","""18""","""003""","""2021-01-10""",58,3,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0


# Convert attributes to factors

- Leave
- College
- Reported satisfaction
- Reported usage level
- Considering change of plan

## Calculate distribution of the outcome  
What is the proportion of people who churned?

Why should we care about this proportion?

1. An important step in EDA is to understand the distribution of the target variable.

2. The majority class in the target variable will serve as an important benchmark for model performance. If we used what we'll call a "majority class classifier"---this consists in always predicting the majority class, which in this case is `STAY`---we would be correct 1 - .49 or 51% of the time.  Another way of saying this is that majority class classifier in the MegaTelCo case would result in accuracy of .51.  

Accuracy is defined as the proportion of correctly predicted labels. It is a commonly used error metric for evaluating classifier performance.

Think about why a majority class model in this case would have an accuracy of .51.

Whatever later model we develop should have better accuracy than this performance benchmark.

In [ ]:
df_clean.head(5)


,"""State Name""","""county Name""","""State Code""","""County Code""","""Date""","""AQI""","""Category""","""Defining Parameter""","""Defining Site""","""Number of Sites Reporting""","""PM2.5""","""Ozone""","""SO2""","""PM10""","""NO2""","""CO"""
54477,"""Indiana""","""Allen""","""18""","""003""","""2021-01-01""",29,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54478,"""Indiana""","""Allen""","""18""","""003""","""2021-01-02""",20,4,"""Ozone""","""18-003-0002""",2,0,1,0,0,0,0
54479,"""Indiana""","""Allen""","""18""","""003""","""2021-01-03""",22,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54480,"""Indiana""","""Allen""","""18""","""003""","""2021-01-04""",31,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0
54481,"""Indiana""","""Allen""","""18""","""003""","""2021-01-05""",43,4,"""PM2.5""","""18-003-0004""",2,1,0,0,0,0,0


## Fit a basic tree model

Use just two variables, 'income' and 'house'. We'll call this the "money tree." 

What is the accuracy of the money tree? 

In [ ]:
# split the datafram into independent (x) and dependent (predicted) attributes (y)
x = df_clean[['"AQI"','"Number of Sites Reporting"','"PM2.5"','"Ozone"','"SO2"','"PM10"','"NO2"','"CO"']]
y = df_clean['"Category"']

category_tree = DecisionTreeClassifier()

# Create Decision Tree Classifer
category_tree = category_tree.fit(x,y)


## Preview the tree


In [ ]:
category_tree_text = tree.export_text(category_tree)
print(category_tree_text)

|--- feature_0 <= 50.50
|   |--- class: 4
|--- feature_0 >  50.50
|   |--- feature_0 <= 100.50
|   |   |--- class: 3
|   |--- feature_0 >  100.50
|   |   |--- feature_0 <= 156.00
|   |   |   |--- class: 2
|   |   |--- feature_0 >  156.00
|   |   |   |--- class: 1



## Check Accuracy

What is the accuracy of the money_tree? Use these steps to calculate accuracy.

Is this over fitted?

In [ ]:
pred = category_tree.predict(x)

#print(pred)

print("Accuracy:",metrics.accuracy_score(y, pred))

Accuracy: 1.0


## Prune the tree

Limit the number of levels to 2

In [ ]:
category_tree2 = DecisionTreeClassifier(criterion="entropy", max_depth=2)

# Create Decision Tree Classifer
category_tree2 = category_tree2.fit(x,y)


## Preview the new tree

In [ ]:
category_tree2_text = tree.export_text(category_tree2)
print(category_tree2_text)

|--- feature_0 <= 50.50
|   |--- class: 4
|--- feature_0 >  50.50
|   |--- feature_0 <= 100.50
|   |   |--- class: 3
|   |--- feature_0 >  100.50
|   |   |--- class: 2



## Re-Check Accuracy

Is this accuracy better than making a random guess?  (check the distribution above)

In [ ]:
pred = category_tree2.predict(x)

print("Accuracy:",metrics.accuracy_score(y, pred))

Accuracy: 0.9998895149707214


## Visualize the model

In [ ]:
from six import StringIO
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(money_tree2, out_file=dot_data, 
                   feature_names=x.columns,class_names=['leave','stay'], 
                   filled=True,rounded=True, precision =2)

graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

## Fit a full tree model

Use all of the independent attributes.  We'll call this the "full tree." 

What is the accuracy of the full tree? 

In [ ]:
# split the datafram into independent (x) and dependent (predicted) attributes (y)
x = df_clean[['income','house','college','overage','leftover','handset_price','over_15mins_calls_per_month','average_call_duration','reported_satisfaction','reported_usage_level','considering_change_of_plan']]
y = df_clean['leave']

full_tree = DecisionTreeClassifier(criterion="entropy", max_depth=1)

# Create Decision Tree Classifer
full_tree = full_tree.fit(x,y)


##Visualize the full tree

In [ ]:
from six import StringIO
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(full_tree, out_file=dot_data, 
                   feature_names=x.columns,class_names=['leave','stay'], 
                   filled=True,rounded=True, precision =2)

graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

## Check Accuracy

In [ ]:
pred = full_tree.predict(x)

#print(pred)

print("Accuracy:",metrics.accuracy_score(y, pred))

##Test and Train
Now we will split the dataset into 80% training data and 20% test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

##Create a new tree using only training data

In [ ]:
train_tree = DecisionTreeClassifier(criterion="entropy", max_depth=5)

# Create Decision Tree Classifer
train_tree = train_tree.fit(x_train,y_train)

##Apply the new tree to our test data

In [ ]:
pred = train_tree.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, pred))

## Visualize the trained tree

In [ ]:
from six import StringIO
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(train_tree, out_file=dot_data, 
                   feature_names=x.columns,class_names=['leave','stay'], 
                   filled=True,rounded=True, precision =2)

graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

## Did the model improve?
👎  👍